In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

#preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.stats import mode


# pipelines
from sklearn.pipeline import Pipeline



# Clustering Customer Data

In this notebook you will use your previously cleaned and preprocessing customer dataset to practice clustering.
I've implemented my own version below (of the preprocessing and cleaning) but you should feel totally free to use your own.



In [ ]:
# Read in the data found in "best_one_ever_database.csv" and take a look at the head and info.
data = pd.read_csv("best_one_ever_database.csv", index_col = 'id')

In [ ]:
data.head()

In [ ]:
data.info()

# Cleaning The Data



In [ ]:
# drop the name columns using pandas.drop()

X = data.drop(labels = ['first_name','last_name'], axis = 1)

In [ ]:
X.info()

### Transform string columns into useful features


In [ ]:
def strip_dollar(x):
    return x[1:]

In [ ]:
# apply the function to the column and assign it back to the column (it does not work inplace)
X.sales = X.sales.apply(strip_dollar)

# cast the column to a floating point type - this is very important, otherwise it will be
# an object type column that we cannot do arithmetic on the column
X.sales = X.sales.astype('float32')

In [ ]:
X.head()

In [ ]:
# define functions to apply to the dataframe
def strip_emails(x):
    at = x.find('@')
    return x[at+1:]

test_email = 'thisismymail@gmail.com'
print(strip_emails(test_email))

In [ ]:
# apply the function to the column and assign it back to the column (it does not work inplace)
X.email = X.email.apply(strip_emails)
X.head()

# Is the email column going to be worth it?
Let's take a look at this email column and decide if it could help us or not.


In [ ]:
# how many unique domains are there?
counts = X.email.value_counts()

In [ ]:
print(counts.values)

No, it doesn't seem like this column will be very helpful as it's quite spread out. There are 490 unique values and no one value has a majority, so let's just leave it to the side for now. We can always come back to it later.

In [ ]:
# we make sure to copy it over and save it for later.
emails = X.email.copy()
X.drop('email', axis=1, inplace = True)

In [ ]:
X.info()

## Splitting the IP Address
We now need to split up the IP address, we will use Pandas's built in str method for this.

In [ ]:
X[['first_ip','second_ip','third_ip','fourth_ip']] = X.ip_address.str.split(pat=".", expand=True)

In [ ]:
# now we cast the columns as floats
X[['first_ip','second_ip','third_ip','fourth_ip']] = X[['first_ip','second_ip','third_ip','fourth_ip']].astype('float32')
# we also drop the original column
X.drop('ip_address', axis=1, inplace=True)

In [ ]:
X.info()

# One Hot Encoding

Ok we are almost done, we just have to convert the gender column into something integer that we can use. We will use one-hot-encoding since gender is a categorical variable.

Pandas has a `get_dummies()` function that will be very useful.


In [ ]:
X.gender.value_counts()

In [ ]:
dumbdumbs = pd.get_dummies(X['gender'])
X= pd.concat((X,dumbdumbs), axis = 1)
X.drop(['gender'], axis = 1, inplace=True)
X.head()

In [ ]:
X.info()

# Data Preprocessing Stage 2 - Missing values



In [ ]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

In [ ]:
X_ = imp.fit_transform(X)

In [ ]:
X= pd.DataFrame(X_, columns=X.columns)

In [ ]:
X.info()

# Clustering

Ok, now we have our customer data all ready to go. We have done all the hard work of preprocessing. Let's feed this data into our algorithms!

We'll try two different clustering algorithms. K-means and DB-scan.


Our goal is to cluster the data and learn what kinds of customers we have, are they related to one another at all? In order to do this we should try some clustering

# Cluster with  K-means

Let's just make some clusters and evaluate it with their silhouette score. A reminder about the silhouette score

>The best value is 1 and the worst value is -1. Values near 0 indicate overlapping clusters. Negative values generally indicate that a sample has been assigned to the wrong cluster, as a different cluster is more similar.

From : https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html#sklearn.metrics.silhouette_score

Your Job:
Run a k-means loop on clusters 2-n (you decide how many you want to try!) and see which # of clusters is best.

You should scale the data first since we are hunting for clusters we definitely want the dimensions to be on the same scale (remember that distance means everything here!).

#### Note:
In an effort to slowly remove your training wheels, I have not important everything you need to accomplish your tasks here.


In [ ]:
# import what you need


In [ ]:
# scale your data with a scaler of your choice


In [ ]:
# create a k-means estimator and fit it.


In [ ]:
# print out the silhouette score


In [ ]:
# run a loop between 2-N (your choice for N!) and print this guy out.

print(f"The number of clusters is {} and the Silhouette score is {}")

### Cluster Discussion

Ok, so we found the best number of clusters according to the silhouette score.  So what? I mean to say, what do we do with that? We know that according the silhouette score this is the best, but... even if that's correct, _now what_? How do we use these clusters to help us run our business?
How can we learn what these clusters represent?

I can think of one thing to check

1. Look at the feature values of the cluster centroids.

Recall that every cluster in kmeans has a centroid. That centroid is the very _center_ of the cluster, so we can look at the feature values of the centroids and see what they tell us. In theory the centroids represent the cluster (generally). 

We can look at the cluster centroids right now.  Let's examine then with a barplot.

I'm going to give you some code help here.

In [ ]:
# fit a kmeans cluster with desired number of components



In [ ]:
# look for the centers of your clusters.  
# Hint: your kmeans object has an attribute you'd want to use



In [ ]:
# my gift to you, use it to plot your centers
def plot_centroids(centers):
    pd.DataFrame(centers, columns = X.columns).plot(kind = 'bar', figsize = (12,6))
    plt.legend(loc='best', bbox_to_anchor=(0.8, 0.1, 0.5, 0.5)); # bbox is (x,y, width, height)


# Examine your centroids

What does your graph tell you?
What features are the most important and contributing towards the clusters?

Extra-Credit: Rerun your clustering algorithm with a _different_ scaling method and re-examine your centroid features. Do they change? Why? What does it mean?

# DB-Scan

Let's now try DB-scan. Remember we have three parameters we need to set
1. `eps` the radius of our circle that we will search in
2. `min_samples` the minimum number of samples we need to find within our radius to call it a cluster
3. `metric` our distance metric.  Defaults to Euclidean (L2-norm).

So, we get to fiddle with 3 parameters, but we don't have to worry about "how many" clusters to look for, db-scan will decide for us.

Go ahead and run it!
What is the best eps / min_samples ?
What gets you the best silhouette score?

In order to answer these questions we will need to figure out a few things

1. run dbscan (that's pretty easy)
2. how many clusters did it find?
3. how do we get the labels (predictions) from the dbscan object?

In [ ]:
# run dbscan 
# pick an eps -- how might we do this? what range are your features in?
# Your eps should be relevant to the feature space you exist in
# pick min_samples




#### ok, now you fit a dbscan. 
1. how do we figure out how many clusters it found?
DBscan has 3 main attributes, 1

>**core_sample_indices_ndarray of shape (n_core_samples,)**
Indices of core samples.

>**components_ndarray of shape (n_core_samples, n_features)**
Copy of each core sample found by training.

>**labels_ndarray of shape (n_samples)**
Cluster labels for each point in the dataset given to fit(). Noisy samples are given the label -1.

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html

Go ahead and look at those attributes. Print them out. Which one will help us figure out how many clusters DBscan found?

Also pay attention to changing `eps`, if `eps` is too small what happens? Too large?

## Calculate the Silhouette Score

In order to perform the silhouette score we need the labels (that's pretty easy they are given to us), however we should _exclude_ the -1's because they represent points that are _not_ in a cluster. So they are basically discarded. We should probably collect those somewhere and else and see how large that number is. But it should not be part of the silhouette core.

So
1. seperate out the points which have the label `-1`

Then after we have done that, we can calcluate the silhouette score easily. 
#### NOTE
DBscan in scikit-learn is implemented with the notion of "core samples".  From the [documentation:](https://scikit-learn.org/stable/modules/clustering.html#dbscan)
>More formally, we define a core sample as being a sample in the dataset such that there exist min_samples other samples within a distance of eps, which are defined as neighbors of the core sample. This tells us that the core sample is in a dense area of the vector space. A cluster is a set of core samples that can be built by recursively taking a core sample, finding all of its neighbors that are core samples, finding all of their neighbors that are core samples, and so on. **A cluster also has a set of non-core samples, which are samples that are neighbors of a core sample in the cluster but are not themselves core samples.** Intuitively, these samples are on the fringes of a cluster.

This means that the attributes `components` and `core_sample_indices` will only return core samples, for us to get all the points _within_ the cluster we will have to rely on the labels

### Number of clusters?
But what about the # of clusters? How many did we find and how many in each cluster? We need to examine the labels again, but this time count the unique ones and also count how many of each unique we have.

### Goal
Run our little line of code, which gives both the score and # of clusters.

`print(f"The number of clusters is {num_clusters} and the Silhouette score is {silhouette_score(x_scaled, dbscan.labels_)}")`


Oh! we also should probably add
`print(f"The number of discarded points was {}")`


In [ ]:
## remove the -1's from the labels.
## you also will need to remove the corresponding x_scaled samples so you can compute the silhouette score.
# you can use numpy boolean indexing masks for this.



In [ ]:
## ok if you have the labels and x's without -1's you can actually computer the Silhouette score now


In [ ]:
## how many clusters are there though?
## you need to figure out how many unique labels there are, and also it would be nice to know
## how many points are in each cluster




### How does it compare?

Did your DBscan find similar clusters to k-means? Same number of clusters? More ? Less? What about the silhouette score? Better or worse?


### Next we'd like to look at the centroids

Actually, DBscan doesn't have centroids naturally the same way k-means does. But we can compute it ourselves.
We would need to isolate the points of each group, and then just take the mean column wise, that would represent the centroid of each cluster.  You can do this with a numpy mask



In [ ]:
# plot your DBScan Centroids



# How do your centroids compare?

Did you find similar clusters? Dissimilar?
